# Notebook BERTopic (Publikationsdaten)

> Dieses ist das erste von zwei Notebooks mit dem gesamten Code zum _topic modeling_ im Bereich der Publikationsdaten.

Das Notebook folgt einem bestimmten Aufbau:
1. Die einzelnen Zellen müssen sequentiell, also in Reihe, durchlaufen werden.
2. Die einzelnen Schritte des Codes werden jeder für sich erläutert und dann aufbgebaut, durch Variablen und Funktionen, sodass nachfolgende Zellen von vorherigen Zellen abhängen.
3. Das Notebook kann dabei zwei verschiedene Ziele verfolgen, (i) Durchführung des _grid search_ und mit dem Ziel der Auswahl eines besten Modells, (ii) Auswertung des _topic modeling_ mit dem vorher eruierten, besten Modell. Es ist darauf zu achten, dass der zweite Schritt nur vollzogen werden kann, wenn der erste zumindest einmal durchgeführt wurde, da auf das gespeicherte, beste Modell zugegriffen werden muss.
  3.1 Um das einzustellen, gibt es in Zelle 3 zwei Variablen, die entsprechend zu steuern sind. Alles Weitere passiert dann automatisch.
4. Da insb. der _grid search_ nicht lokal durchgeführt werden konnte, gibt es in der ersten Zelle eine automatische Umgebungserkennung, die registrieren kann, ob man lokal oder in Google Colab arbeitet. In Google Colab wurde mit gekauften Recheneinheiten verfahren und einer A100 gearbeitet.

> Abschließend ist noch anzumerken, dass alle Stellen, die mit der Hilfe moderner KI-Unterstützung überarbeitet wurden, z. B. aufgrund von Fehlerhaftigkeit oder schlechter Funktionalität, als solche gekennzeichnet sind (zum Suchen: "**[KI]**").

## Festlegung der _Pipeline_ (_Grid Search_ oder Modellauswertung)

Im Folgenden muss festgelegt werden, welche Pipeline genommen werden soll.

In [ ]:
##############################################################################
##############################################################################
grid_search_pipeline = False
evaluation_pipeline = True if grid_search_pipeline is False else False

using_all_models = True if grid_search_pipeline is True else False  # Wenn True, werden alle Modelle genutzt, die für das Grid Search gebraucht werden! Embeddings werden neu erstellt und gespeichert!
using_top_models = False if grid_search_pipeline is True else True  # Wenn True, werden nur die beiden besten Modelle genutzt, basierend auf der Grid-Search-Auswertung! Embeddings werden geladen!
##############################################################################
##############################################################################

print(f"Grid Search: {grid_search_pipeline}\nEvaluation: {evaluation_pipeline}\n"
f"Alle Modelle einbeziehen: {using_all_models}\nNur beste Modelle: {using_top_models}")

In [ ]:
# Diese erste Zeile prüft automatisch, ob in Colab oder lokal gearbeitet wird und setzt die Pfade
# entsprechend zu den unterschiedlichen Verzeichnissen! Außerdem importiert sie die wichtigsten
# Module!

from pathlib import Path
import os
import sys

# Erkennungsvariable als bool setzen
colab_active = "google.colab" in sys.modules

# print(colab_active)

if colab_active is True:
    print("Notebook befindet sich in Colab-Env. Drive wird gemountet und die nötigen"
    " Pakete werden installiert.")

    print("\nDrive wird verbunden.")
    from google.colab import drive
    drive.mount("/content/drive")
    base_dir = Path("/content/drive/Othercomputers/laptop/masterarbeit")

    # cuML GPU Beschleunigung: Import der anderen UMAP und HDBSCAN Module!
    # (Quellen:
    # https://odsc.medium.com/accelerating-umap-processing-10-million-records-in-under-a-minute-with-no-code-changes-5d580deb05a7
    # https://docs.rapids.ai/api/cuml/stable/
    # https://docs.rapids.ai/install/#selector)

    print("\ncuML-Pakete werden importiert.")
    from cuml.manifold import UMAP
    from cuml.cluster import HDBSCAN

    # Installation weiterer Packages für das Colab-Environment
    print("\nInstallation weiterer Pakete.")
    %pip install -q bertopic sentence-transformers hdbscan anthropic --upgrade gensim litellm

    # Pfade werden definiert
    data_dir_pubs = base_dir / "01_data" / "01_csv_data" / "99_pubmed"  # nur PubMed!
    data_dir_tpf = base_dir / "01_data" / "01_csv_data"                 # hier liegen tpf Daten

    embedds_dir = base_dir / "01_data" / "03_topic_modeling" / "01_embeddings"
    topic_results_dir = base_dir / "01_data" / "03_topic_modeling" / "02_topic_results"
    topic_visuals_dir = base_dir / "01_data" / "03_topic_modeling" / "03_topic_visuals"
    grid_search_dir = base_dir / "01_data" / "03_topic_modeling" / "04_grid_search"
    models_dir = base_dir / "01_data" / "03_topic_modeling" / "05_models"

    # Check auf erreichbare Pfade
    all_paths = [base_dir, data_dir_pubs, data_dir_tpf, embedds_dir, topic_results_dir,
                 topic_visuals_dir, grid_search_dir, models_dir]

    no_paths = [x for x in all_paths if not x.exists()]

    if no_paths:
        print(f"Folgende Pfade konnten nicht erreicht werden:\n{no_paths}.")
    else:
        print("\nAlle Pfade konnten gesetzt und gefunden werden!")

else:
    print("Notebook ist lokal! Pfade werden entsprechend der Ordnerstruktur"
          " geprüft und ggf. gesetzt.")

    # Import der nicht-GPU-optimierten Module von UMAP und HDBSCAN
    print("\nImport der UMAP- und HDBSCAN-Module (nicht-GPU-optimiert).")
    from umap import UMAP
    from hdbscan import HDBSCAN

    # Ordnerpfade definieren
    print("\nAufsetzen und Prüfung der Ordnerpfade in lokaler Umgebung.")
    base_dir = Path.cwd().parent # entspricht dem Ordner "masterarbeit"

    # Pfade werden definiert
    data_dir_pubs = base_dir / "01_data" / "01_csv_data" / "99_pubmed"  # nur PubMed!
    data_dir_tpf = base_dir / "01_data" / "01_csv_data"                 # hier liegen tpf Daten

    embedds_dir = base_dir / "01_data" / "03_topic_modeling" / "01_embeddings"
    topic_results_dir = base_dir / "01_data" / "03_topic_modeling" / "02_topic_results"
    topic_visuals_dir = base_dir / "01_data" / "03_topic_modeling" / "03_topic_visuals"
    grid_search_dir = base_dir / "01_data" / "03_topic_modeling" / "04_grid_search"
    models_dir = base_dir / "01_data" / "03_topic_modeling" / "05_models"

    # Check auf erreichbare Pfade
    all_paths = [base_dir, data_dir_pubs, data_dir_tpf, embedds_dir, topic_results_dir,
                 topic_visuals_dir, grid_search_dir, models_dir]

    no_paths = [x for x in all_paths if not x.exists()]

    if no_paths:
        print(f"\nFolgende Pfade konnten nicht erreicht werden:\n{no_paths}.")
    else:
        print("\nAlle Pfade konnten gesetzt und gefunden werden!")

## Alle Importe und Laden der _Transformer Model_

Folgend werden alle notwendigen Importe durchgeführt. Außerdem werden die verschiedenen Transformer-Modelle in einer Dataclass geladen -- je nach Pipeline und Umgebung.

In [ ]:
# from pathlib import Path
from typing import Any, Annotated, Callable, Iterable
import pandas as pd
import re
import openpyxl
import os
import datetime
import time
import traceback
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from dataclasses import dataclass
from dataclasses import asdict
try:
    import kaleido
except Exception as e:
    print(f"Fehler im Modul Kaleido...: {e}. Der Code kann trotzdem durchlaufen,"
    " es werden nur keine png-Exporte erstellt.")

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired
import openai
from bertopic.representation import OpenAI
from bertopic.representation import TextGeneration
# from crewai import LLM
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

print("Pakete wurden importiert.")

# NLTK Wordnet Check und Load sowie Setzen der Environ_variables
if not colab_active:
    try:
        nltk_path = Path(r"C:\Users\felix\AppData\Roaming\nltk_data\corpora\wordnet.zip")
        punkt_path = Path(r"C:\Users\felix\AppData\Roaming\nltk_data\tokenizers\punkt_tab")

        if nltk_path.exists():
            print("Wordnet-Datei ist schon vorhanden und muss nicht erneut geladen werden!")
        else:
            print("Das NLTK-Paket konnte nicht gefunden werden und wird jetzt geladen!")
            nltk.download("wordnet")

        if punkt_path.exists():
            print("Punkt-Datei ist schon vorhanden und muss nicht erneut geladen werden!")
        else:
            print("Das Punkt-Paket konnte nicht gefunden werden und wird jetzt geladen!")
            nltk.download("punkt_tab")

        # API KEYs
        try:
            MY_API_KEY = os.environ.get("openaikey1")
            if MY_API_KEY is None:
                print("Der OpenAI-API Key wurde nicht gefunden.")
            else:
                print("Der OpenAI-API Key wurde geladen.")

        except Exception as e:
            print(f"Fehler beim Laden der API Keys: {e}.")

    except Exception as e:
        print(f"Es kam zu einem Fehler: {e}.")

elif colab_active:
    print("Das NLTK- und Punkt-Paket wird geladen.")
    nltk.download("wordnet")
    nltk.download("punkt_tab")

    # API Key
    MY_API_KEY = None

#########################################################################################################
# Alle wichtigen Vorbereitungsschritte werden hier schon ausgeführt:
#########################################################################################################

# Tagesdatum aktuell festlegen
datum = datetime.datetime.now().strftime("%d.%m.%y")

# Dataclass nutzen, um Sentence-Transformer-Modelle zu organisieren
# (Quelle: https://www.datacamp.com/tutorial/python-data-classes)
@dataclass
class Models:
    raw_instance: SentenceTransformer
    name: str
    trained_instance: object | None = None
    embeddings: np.ndarray | None = None
    doc_topics_assignment: list[int] | None = None
    final_topics_df: pd.DataFrame | None = None
    umap_n_neighbors: int | None = None
    hdbscan_min_cluster_size: int | None = None
    hdbscan_min_samples: int | None = None
    vectorizer_min_df: int | None = None
    vectorizer_max_df: float | None = None

if using_all_models is True and using_top_models is True:
    raise ValueError("\nEs kann nur eine der beiden Optionen auf True gesetzt werden!")
    sys.exit(1)

if colab_active is True:
    if using_all_models is True:

        print("\nColab ist aktiv -- Und: es werden alle Modelle (nicht nur die besten) geladen!")

        # Erstes Dict für die Publikationen
        model_dict = {
            "all-MiniLM": Models(SentenceTransformer("all-MiniLM-L6-v2", device="cuda"), str("all-MiniLM")),
            "mpnet-base": Models(SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device="cuda"), str("mpnet-base")),
            "pubmed": Models(SentenceTransformer("pritamdeka/S-PubMedBert-MS-MARCO", device="cuda"), str("pubmed")),
            "specter": Models(SentenceTransformer("sentence-transformers/allenai-specter", device="cuda"), str("specter"))
        }

        # Zweites Dict für Drittmittel
        # model_dict_tpf = {
        #     "allgemein_ml": Models(SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2", device="cuda"), str("allgemein_ml")),
        #     "allgemein_ml_2": Models(SentenceTransformer("intfloat/multilingual-e5-large", device="cuda"), str("allgemein_ml_2")),
        #     "allgemein_ml_3": Models(SentenceTransformer("sentence-transformers/LaBSE", device="cuda"), str("allgemein_ml_3"))
        # }

    elif using_top_models is True:

        print("\nColab ist aktiv -- Und: es werden nur die besten Modelle geladen!")

        # Nur die beiden besten Modelle -- diese müssen manuell zugewiesen werden!
        top_model_dict = {
            "all-MiniLM": Models(SentenceTransformer("all-MiniLM-L6-v2", device="cuda"), str("all-MiniLM"))
        }

        print(80*"=")

        # Trainierte Modelle laden, falls vorhanden
        if models_dir.exists():
            for x,y in top_model_dict.items():
                print(f"\nStart der Suche nach dem letzten Modellordner für das Modell {y.name}.")
                try:
                    folders = [f.name for f in models_dir.iterdir()]
                    print(f"Liste der Modell-Ordner:\n{folders}")
                    model_folder = [f for f in folders if y.name+"_" in str(f) and "publications" in str(f)]
                    print(f"Ausgewählter Modellordner:\n{model_folder}")

                    if model_folder:
                        latest_folder_name = max(model_folder, key=lambda f: (models_dir / f).stat().st_mtime)
                        latest_model_path = models_dir / latest_folder_name
                        print(f"\nTrainiertes Modell für {y.name} wurde gefunden und wird geladen.")
                        y.trained_instance = BERTopic.load(latest_model_path)
                    else:
                        print(f"\nKein trainiertes Modell im Ordner \"{models_dir}\" für {y.name} gefunden.")

                except Exception as e:
                    print(f"\nFehler beim Finden des neuesten {y}-Modellordners: {e}")

        else:
            print(f"\nDer Modelle-Ordner \"{models_dir}\" konnte nicht gefunden werden.")

        # Document-Topic-Assignment laden, falls vorhanden
        if topic_results_dir.exists():
            for x,y in top_model_dict.items():
                print(f"\nStart der Suche nach der letzten Doc-Topic-Datei für das Modell {y.name}.")
                try:
                    files = [f.name for f in topic_results_dir.glob("*")]
                    print(f"Liste der Doc-Topic-Dateien:\n{files}")
                    file_selection = [f for f in files if y.name+"_" in str(f) and "publications" in str(f)]
                    print(f"Dateiauswahl:\n{file_selection}")

                    if file_selection:
                        latest_file_name = max(file_selection, key=lambda f: (topic_results_dir / f).stat().st_mtime)
                        latest_file_path = topic_results_dir / latest_file_name
                        print(f"\nDocument-Topic-Assignment für {y.name} wurde gefunden und wird geladen.")
                        y.doc_topics_assignment = pd.read_csv(latest_file_path, encoding="utf-8")["topics"].tolist()
                    else:
                        print(f"\nKeine Doc-Topic-Datei im Ordner \"{topic_results_dir}\" für {y.name} gefunden.\n")

                except Exception as e:
                    print(f"\nFehler beim Finden der letzten {y}-Doc-Topic_File: {e}")

        else:
            print(f"\nDie Doc-Topic_Datei in \"{topic_results_dir}\" konnte nicht gefunden werden.")


elif colab_active is False:
    if using_all_models is True:

        print("\nCode läuft lokal -- Und: es werden alle Modelle (nicht nur die besten) geladen!")

        # Erstes Dict für die Publikationen
        model_dict = {
            "all-MiniLM": Models(SentenceTransformer("all-MiniLM-L6-v2"), str("all-MiniLM")),
            "mpnet-base": Models(SentenceTransformer("sentence-transformers/all-mpnet-base-v2"), str("mpnet-base")),
            "pubmed": Models(SentenceTransformer("pritamdeka/S-PubMedBert-MS-MARCO"), str("pubmed")),
            "specter": Models(SentenceTransformer("sentence-transformers/allenai-specter"), str("specter"))
        }

        # Zweites Dict für Drittmittel
        # model_dict_tpf = {
        #     "tpf1": Models(SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2"), str("tpf1")),
        #     "tpf2": Models(SentenceTransformer("intfloat/multilingual-e5-large"), str("tpf2")),
        #     # "tpf3": Models(SentenceTransformer("LaBSE"), str("tpf3"))
        # }

    elif using_top_models is True:

        print("\nCode läuft lokal -- Und: es werden nur die besten Modelle geladen!\n")

        # Nur die beiden besten Modelle
        top_model_dict = {
            "all-MiniLM": Models(SentenceTransformer("all-MiniLM-L6-v2"), str("all-MiniLM"))
        }

        print(80*"=")

        # Trainierte Modelle laden, falls vorhanden
        if models_dir.exists():
            for x,y in top_model_dict.items():
                print(f"\nStart der Suche nach dem letzten Modellordner für das Modell {y.name}.")
                try:
                    folders = [f.name for f in models_dir.iterdir()]
                    print(f"Liste der Modell-Ordner:\n{folders}")
                    model_folder = [f for f in folders if y.name+"_" in str(f) and "publications" in str(f)]
                    print(f"Ausgewählter Modellordner:\n{model_folder}")

                    if model_folder:
                        latest_folder_name = max(model_folder, key=lambda f: (models_dir / f).stat().st_mtime)
                        latest_model_path = models_dir / latest_folder_name
                        print(f"\nTrainiertes Modell für {y.name} wurde gefunden und wird geladen.\n")
                        y.trained_instance = BERTopic.load(latest_model_path)
                    else:
                        print(f"\nKein trainiertes Modell im Ordner \"{models_dir}\" für {y.name} gefunden.\n")

                except Exception as e:
                    print(f"\nFehler beim Finden des neuesten Modellordners für das Modell {y.name}: {e}.\n")

        else:
            print(f"\nDer Modelle-Ordner \"{models_dir}\" konnte nicht gefunden werden.")

        print(80*"=")

        # Document-Topic-Assignment laden, falls vorhanden
        if topic_results_dir.exists():
            for x,y in top_model_dict.items():
                print(f"\nStart der Suche nach der letzten Doc-Topic-Datei für das Modell {y.name}.")
                try:
                    files = [f.name for f in topic_results_dir.glob("*")]
                    print(f"Liste der Doc-Topic-Dateien:\n{files}")
                    file_selection = [f for f in files if y.name+"_" in str(f) and "publications" in str(f) and "doc_topics_assignment_" in str(f)]
                    print(f"Dateiauswahl:\n{file_selection}")

                    if file_selection:
                        latest_file_name = max(file_selection, key=lambda f: (topic_results_dir / f).stat().st_mtime)
                        latest_file_path = topic_results_dir / latest_file_name
                        print(f"\nDocument-Topic-Assignment für {y.name} wurde gefunden und wird geladen.\n")
                        y.doc_topics_assignment = pd.read_csv(latest_file_path, encoding="utf-8")["topics"].tolist()
                    else:
                        print(f"\nKeine Doc-Topic-Datei im Ordner \"{topic_results_dir}\" für {y.name} gefunden.\n")

                except Exception as e:
                    print(f"\nFehler beim Finden der letzten Doc-Topic_File für das Modell {y.name}: {e}.\n")

        else:
            print(f"\nDie Doc-Topic_Datei in \"{topic_results_dir}\" konnte nicht gefunden werden.\n")

        print(80*"=")


## Vorverarbeitung der Daten

Die Publikationsdaten werden eingelesen und vorverarbeitet.

In [ ]:
#########################################################################################################
# Datenvorverarbeitung der Publikationsdaten
#########################################################################################################

# Ordner einlesen, in die Dateien liegen
folder = data_dir_pubs

# Einzelne Csv-Dateien in Liste packen
concat_list = [pd.read_csv(x, encoding="utf-8") for x in folder.glob("*.csv")]

#print(concat_list)

# Finalen Dataframe aller Publikationen konkatenieren
df_all = pd.concat(concat_list, axis=0, ignore_index=True)

# Check
print(80*"=")
print("Shape des Gesamtdf:")
print(df_all.shape)
print(80*"=")
print("Spalten des Gesamtdf:")
print(df_all.columns)
print(80*"=")
print("Übersicht zu den Werten:")
print(df_all.info())
print(80*"=")

#########################################################################################################
# Erstes Preprocessing der Titel+Abstract-Strings
#########################################################################################################

# Leere und unvollständige Titel u/o Abstracts entfernen
df_all = df_all.dropna(subset=["abstract", "title"])
df_all = df_all[~(df_all["abstract"] == "no abstract")]

# Titel und Abstracts aufteilen
docs_titles = df_all["title"] #.tolist()
docs_abstracts = df_all["abstract"]# .tolist()
#docs_keywords = df_all["keywords"].str.replace("[", "").str.replace("]", "") #.tolist()

# Kurze Überprüfungen
#print([len(x) for x in [docs_titles, docs_abstracts, docs_keywords]])
#print(len(df_all[df_all["abstract"].str.lower().isin(["no"])]))
#print(docs_abstracts)

# Erstellung der Dokumentenliste
processed_docs = [title + "|" + abstract for title, abstract in zip(docs_titles, docs_abstracts)]

# Check
print("Anzahl der Einträge der processed docs nach dem Entfernen von Leerzeilen bzw. falsch befüllten Zeilen:")
print(len(processed_docs))
print(80*"=")

In [ ]:
# Die folgenden Funktion ist ein kurzes, textuelles Preprocessing der Titel+Absttract-Strings
# (Anm.: Die Dokumentation erwähnt explizit, dass die Stopwörter hier noch nicht entfernt
# werden sollen. Das erfolgt erst im eigentlichen Modellaufruf durch ein CountVectorizer-Modell)

def preprocessed_text(text: str) -> str:
    """Die Strings aus Titeln und Abstracts werden leicht vorverarbeitet mit üblichen Stringoperationen und Regexfunktionen.

    Args:
    - Text als String

    Returns:
    - Bearbeiteten Text
    """

    text = text.lower()
    text = re.sub(r"[^\w\s|]", "", text)
    text = re.sub(r"\s+", " ", text).strip()

    return text

# Funtkionsaufruf für jedes Item aus preprocessed_docs
cleaned_docs = [preprocessed_text(x) for x in processed_docs]

# Check
def preprocessing_check(processed_list: list[str], realm: Annotated[str, "Entweder Publikationen oder Drittmitteldaten"]):
    """Diese kleine Funktion checkt die Ergebnisse der Vorverarbeitungen der Textdaten.

    Args:
    -

    Returns
    -
    """

    print(f"Anzahl der {realm} (Titel+Abstract) der vorverarbeiteten Daten nach dem Preprocessing:")
    print(len(processed_list))
    print(80*"=")
    print(f"Anzahl der Wörter/Token der vorverarbeiteten Daten ({realm}) nach dem Preprocessing:")
    list_of_words = []
    for x in processed_list:
        b = x.split(" ")
        list_of_words.append(b)
    print(sum([sum([1 for x in y]) for y in list_of_words]))
    print(80*"=")

In [ ]:
# Funtkionsaufruf für den Check
preprocessing_check(cleaned_docs, "Publikationen")

## Fuktionsdefinition der Embeddings

Nachdem die Vorverarbeitung bereinigte, nutzbare Daten erzeugt hat, können diese zur Erstellung der Vektorembeddings genutzt werden.

In [ ]:
# Die folgende Funktion erstellt ODER lädt bestehende Embeddings der drei verwendeten Modelle
# (Anm.: Da das Erstellen der Embeddings einige Zeit in Anspruch nimmt, wurden diese gespeichert und werden jedes Mal wieder geladen,
# wenn man es so einstellt bzw. sie vorhanden sind. Dafür ist allerdings auch die richtige Ordnerstruktur entscheidend, damit diese
# gespeichert und geladen werden können. )


def create_or_load_embeddings(realm: Annotated[str, "Entweder 'publications' oder 'tpf'"],
                              docs: list[str],
                              model_dict: dict,
                              # model_names: list[str] = [b.name for _,b in model_dict.items()],
                              load_embeds: bool = True) -> np.ndarray:
    """
    Diese Funktion erstellt die Embeddings der Titel+Abstract-Kombinationen für Publikationen und Drittmitteldaten mit vier verschiedenen
    Sentence-Transformer-Modellen:

    1. Allgemeines Modell
    2. PubMed-Modell
    3. SciBERT
    4. SPECTER2

    Dafür werden nur die folgenden Argumente gebraucht:

    Args:
    - realm = "publications" oder "tpf"
    - docs =
    - model_dict =
    - load_embedds =

    Returns:
    - es wird direkt nichts ausgegeben, wohl aber werden die Embeddings in der jeweiligen Model-Class gespeichert!

    """

    ###############################################
    # 1. Teil: Embeddings **laden**
    ###############################################
    # Wenn Arg "load_embeds" == True (Standardwert), dann werden keine Embeddings erstellt, sondern lokal geladen! Es wird automatisch die
    # zuletzt erstelle Datei gesucht

    if load_embeds is True:
        # Hier werden zunächst die jeweils *letzten* Embeddings identifiziert
        embedding_folder_files = list(Path(embedds_dir).glob("*.npy"))

        # Schleife für alle Modelle
        for x,y in model_dict.items():#
            try:
                files = [x for x in embedding_folder_files if y.name + "_" in str(x).lower()]
                latest_file = max(files, key=lambda x: x.stat().st_mtime, default=None)

                model_dict[x].embeddings = np.load(latest_file)

                print(f"Name der zuletzt gespeicherten Embeddings aus dem Modell \"{y.name}\": {latest_file}.")
                print(f"Prüfung der Anzahl der Embeddings von {y.name} und ihrer Dimensionalität:")
                print(f"{y.embeddings.shape[0]} Datensätze mit {y.embeddings.shape[1]} Dimensionen.")
                print(80*"=")

            except Exception as e:
                print(f"Es konnte keine letzte Datei mit Embeddings zu diesem Modell ({y.name}) gefunden werden.")
                return False

    ###############################################
    # 2. Teil: Embeddings **erzeugen** und speichern (wenn so angegeben!)
    ###############################################
    # Wenn Arg "load_embeds" == False, dann werden keine Embeddings erstellt, sondern geladen.
    # Das Erzeugen der Embeddings dauert je nach Kapazitäten und Modellen bis zu 30 Minuten

    if load_embeds is False:

        # Die Embedding-Objekte sind schon in der ersten Zelle definiert
        for x,y in model_dict.items():

            try:
                # Embeddings werden erstellt
                print(f"Modell \"{y.name}\" startet.")
                start = time.time()
                embeddings = y.raw_instance.encode(docs, batch_size=128, show_progress_bar=True)
                end = time.time()
                print("Die Embeddings mit {} sind mit einer Laufzeit von {} Minuten erstellt worden.\n".format(y.name, int((end-start)/60)))

                # Embedings werden gespeichert
                embeddings_df = pd.DataFrame(embeddings)
                embeddings_df.to_excel(rf"{embedds_dir}/embeddings_{realm}_{y.name}_{datum}.xlsx",
                                    sheet_name="embeddings", engine="openpyxl")
                np.save(rf"{embedds_dir}/embeddings_{realm}_{y.name}_{datum}.npy", embeddings)

                # Embeddings zuweisen
                y.embeddings = embeddings

                # Überprüfung
                print(f"Prüfung der Anzahl der Embeddings von {y.name} und ihrer Dimensionalität:")
                print(f"{y.embeddings.shape[0]} Datensätze mit {y.embeddings.shape[1]} Dimensionen.")
                print(80*"=")

            except Exception as e:
                print(f"Fehler beim Modell {y.name}: {e}.")

In [ ]:
##########################################
# Funktion zum Check auf lokale Embeddings
##########################################

def check_local_embeddings(model_dict: dict) -> bool:
    """Diese Funktion checkt, ob für alle Modelle im übergebenen Dict Embeddings
    lokal vorhanden sind.

    Args:
    - model_dict = Dict mit den Modellen

    Returns:
    - bool: True, wenn für alle Modelle Embeddings vorhanden sind, sonst False
    """

    all_exist = True

    for x,y in model_dict.items():
        if y.embeddings is None:
            print(f"Für das Modell {y.name} sind keine lokalen Embeddings vorhanden.")
            all_exist = False
        else:
            print(f"Für das Modell {y.name} sind lokale Embeddings vorhanden.")

    return all_exist

# Funktionsaufruf
# check_local_embeddings

In [ ]:
##########################################
# Funktionsaufruf zum ERSTELLEN der Embeddings für die Publikationen
##########################################

if using_all_models is True:    # Dieser Weg führt zum Grid Search, Embeddings werden neu erstellt und gespeichert
    print("Es werden alle Modelle genutzt, für die die Embeddings **neu** erstellt werden.\n")
    create_or_load_embeddings(realm="publications", docs=cleaned_docs, model_dict=model_dict, load_embeds=False)
else:
    print("Embeddings werden hier nicht erstellt, da die Auswertung der besten Modelle folgt.\n")

# elif using_top_models is True:  # Dieser Weg führt zur Auswertung, Embeddings werden nur noch geladen
#     print("Es werden nur die besten Modelle genutzt, für die die Embeddings geladen werden.\n")
#     create_or_load_embeddings(realm="publications", docs=cleaned_docs, model_dict=top_model_dict, load_embeds=True)
#     print(f"Check der geladenen Embeddings für die besten Modelle:\n{[top_model_dict[x].embeddings.shape for x in top_model_dict.keys()]}")


## Funktionserstellung zur Durchführung des _Topic Modeling_

Folgend wird eine Funktion definiert, die das Themenclustering durchführt. Dafür werden zunächst die Stopwörter definiert und weitere Vorbereitungsschritte durchgeführt.

In [ ]:
# Da sich in verschiedenen Testläufen die Entfernung der Stopwörter durch die Standardstopwörterliste als ungeeignet erwiesen hat, weil immer wieder eine
# große Anzahl von ihnen in den Topic-Clustern erschienen ist, mussten die Stopwörter umfassend manuell erweitert werden. Integriert sind jetzt all diejenigen,
# die standardmäßig in hoher Anzahl in (natur-)wissenschaftlichen, englischen Texten vorkommen und das Topic-Modeling dahingehend beeinflussen.

# Erweiterte Stop-Wörterliste
# (Quellen: Eigene Analyse der Topic-Cluster-Ergebnisse sowie
# https://www.ranks.nl/stopwords und in der finalen Überarbeitung
# eine Überprüfungs- und Ergänzungsanfrage bei ChatGPT [KI])
custom_additions = [
    "the", "and", "of", "in", "to", "is", "for", "was", "we", "that",
    "with", "by", "as", "are", "this", "it", "from", "on", "an", "be",
    "were", "which", "or", "at", "can", "been", "has", "have", "had",
    "they", "their", "these", "those", "than", "then", "them", "there",
    "when", "where", "who", "will", "would", "should", "could", "may",
    "might", "must", "our", "my", "your", "its", "his", "her", "into",
    "through", "during", "before", "after", "above", "below", "up", "down",
    "out", "off", "over", "under", "again", "further", "once", "here",
    "also", "such", "only", "own", "same", "so", "than", "too", "very",
    "can", "just", "don", "now", "use", "using", "used"
]

# Weitere spezifische Stopwörter
academic_stopwords = [
    "study", "studies", "research", "article", "paper", "results", "data",
    "analysis", "methods", "method", "approach", "examined", "investigated",
    "findings", "conclusion", "conclusions", "present", "presented",
    "show", "showed", "shown", "demonstrate", "demonstrated", "found",
    "observed", "reported", "compared", "based", "studied", "analyzed",
    "identified", "examined", "evaluated", "assessed", "determined",
    "associated", "related", "significant", "ignificantly", "effects",
    "effect", "between", "among", "across", "within", "using", "used",
    "experiment", "experiments", "sample", "samples", "population",
    "participants", "subjects", "variables", "variable", "measured",
    "measurement", "results", "conclusions", "implications", "limitations",
    "future", "directions", "introduction", "background", "literature",
    "review", "theory", "theoretical", "framework", "model", "models"
]

# Zusammenführen der Stop-Wörter in einer Liste
comprehensive_stopwords = list(ENGLISH_STOP_WORDS) + custom_additions + academic_stopwords


# Lemmatization hinzufügen, um die KEywords der Topic Cluster zu verbessern, v.a. um doppelte Wörter zu vermeiden!
# (Quelle: Dokumentation BERTopic unter: https://github.com/MaartenGr/BERTopic/issues/286)
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

###########################################################################################################
# Funktion, um BERTopic zu initialisieren und das Clustering durchzuführen
###########################################################################################################

def topic_clustering(realm: str, sentence_transformer: SentenceTransformer, docs: list[str], embeddings: np.ndarray, hdbscan_min_cluster_size: int = 20, hdbscan_min_samples: int = 5,
                     umap_n_neighbors: int = 15, vec_max_df: float = 0.95, vec_min_df: int = 2, stop_words: list = comprehensive_stopwords, save_xlsx: bool = False,
                     model_nr_topics: int | None = None, ai_model: str | None = None, model_name: str | None = None, datum: str | None = None, save_model: bool = False):
    """
    Diese Funktion führt ein Topic Modeling auf Basis von vorbearbeiteten Textdaten sowie bereits erstellen Embeddings durch und nimmt zudem
    eine Liste mit Stop-Wörtern und Hyperparametern zu den Teilmodulen UMAP und HDBSCAN entgegen.

    Args:
    - embedding_model = Hier wird das gewählte Embedding Model übergeben.
    - docs = Hier werden die Datensätze in Form einer Liste übergeben.
    - embeddings = Hier werden die vorkalkulierten Embeddings in einem Array übergeben.
    - hdbscan_min_cluster_size = Hier kann die Anzahl der Topics variiert werden (um die Granularität einzustellen). Standardmäßig ist der in dieser Arbeit beste Wert voreingestellt.
    - umap_n_neighbors = Legt wiederum die Anzahl der Clustergrößen fest
    - model_nr_topics = Hier kann man die Anzahl der Topics im Output direkt festlegen lassen, wenn man bspw. weiß, wie viele es sein sollen.
    - stop_words = Hier wird die erweiterte Stopwörterliste übergeben.

    Returns:
    - Einen Dataframe mit den Ergebnissen des Topic Clusterings.
    - Das trainierte Topic Model.
    """

    # Zeit nehmen
    start_time = time.time()

    try:
        # UMAP zur Dimensionsreduktion mit den Paramtern:
        umap_model = UMAP(n_neighbors=umap_n_neighbors, # legt die Anzahl der Clustergröße fest
                        n_components=5, # legt die Anzahl der Zieldimension fest, auf die reduziert werden soll
                        min_dist=0.0, # legt den Abstand der dimensionreduzierten Clusterpunkte im Raum fest (Unschärfe)
                        metric="cosine", # Metrik zur Bestimmung der Abweichungen/Distanzen
                        random_state=42 # Sorgt für die Reproduzierbarkeit der Ergebnisse
                        )
    except Exception as e:
        print(f"Fehler im UMAP-Algorithmus für das Model {model_name}: {e}.")
        pass

    try:
        # HDBSCAN zum Clustering der Dokumente mit den Parametern
        hdbscan_model = HDBSCAN(min_cluster_size=hdbscan_min_cluster_size, # Minimum an Dokumenten für ein Cluster
                                min_samples=hdbscan_min_samples, # Dichte der Elemente eines Clusters
                                metric="euclidean", # Standardmetrik, passend für wenige Dimensionen
                                cluster_selection_method="eom", # Clusterauswahl; Alternative: leaf
                                prediction_data=True # ermöglicht neue Vorhersagen für neue Dokumente
                            )
    except Exception as e:
        print(f"Fehler im HDBSCAN-Algorithmus für das Model {model_name}: {e}.")
        pass

    try:
        # Vec-Model für stop-word-removal und Erstellung von N-Grammen mit den Parametern:
        vectorizer_model = CountVectorizer(stop_words=stop_words, # Übergabe der definierten Stopwörterliste
                                        tokenizer=LemmaTokenizer(),
                                        min_df=vec_min_df, # wie oft ein Term in den Dokumenten vorkommen **muss**
                                        max_df=vec_max_df, # wie oft ein Term in den Dokumenten vorkommen **darf**
                                        ngram_range=(1, 2), # Angabe der N-Gramme (hier: Ein- bis einschl. Zwei-Wort-Paare)
                                        token_pattern=r"\b[a-zA-Z]{3,}\b" # Akzeptiert werden nur Wörter mit mind. 3 Buchstaben
                                        )
    except Exception as e:
        print(f"Fehler im CountVectorizer für das Model {model_name}: {e}.")
        pass

    # Representation Model (Empfehlung aus der Dokumentation) -- kann man mit OpenAI machen, muss man aber nicht

    # gpt_5_1 = LLM(
    #         model="gpt-5.1",
    #         drop_params=True,
    #         additional_drop_params=["stop"]
    #     )

    representation_model = None
    if ai_model is None:
        print("Es wird kein LLM für die Repräsentationen verwendet!")
        representation_model = KeyBERTInspired()
    elif ai_model == "open_ai":
        print("\nOpenAI soll für das Representation Model genutzt werden.")
        if MY_API_KEY:
            print("\nAPI-Key konnte gefunden werden!")
            client = openai.OpenAI(api_key=MY_API_KEY)

            # Erstellung der Prompts nach Dokumentation von Grootendorst!
            summarization_prompt = """
            I have a topic that is described by the following keywords: [KEYWORDS]
            In this topic, the following documents are a small but representative subset of all documents in the topic:
            [DOCUMENTS]

            Based on the information above, please give a description of this topic in the following format:
            topic: <description>
            """
            title_prompt= """
            I have a topic that contains the following documents:
            [DOCUMENTS]
            The topic is described by the following keywords: [KEYWORDS]

            Based on the information above, extract a short topic label in the following format:
            topic: <topic label>
            """

            #Modeldefinition
            representation_model = {
                "Main": KeyBERTInspired(),
                "ChatGPT": OpenAI(client, model="gpt-4o",
                                  prompt=summarization_prompt,
                                  nr_docs=5,
                                  delay_in_seconds=10 #,
                                  #generator_kwargs={"stop": None}
                                  ),
                "ChatGPT_titles": OpenAI(client, model="gpt-4o",
                                         prompt=title_prompt,
                                         nr_docs=5,
                                         delay_in_seconds=4)
            }
        else:
            print("Kein OpenAI-API-Key vorhanden! KeyBERTInspired wird genommen!")
            representation_model = KeyBERTInspired()

    if representation_model is None:
        raise ValueError("Aufgrund eines Fehlers wurde kein Representation Model geladen!")

    try:
        # BERTopic starten mit den Parametern:
        topic_model = BERTopic(embedding_model=sentence_transformer,        # Sentence-Transformer-Modell
                            umap_model=umap_model,                          # Modell zur Dimensreduktion
                            hdbscan_model=hdbscan_model,                    # Clusteringmodell
                            vectorizer_model=vectorizer_model,              # Vectorizer
                            representation_model=representation_model,      # zwei Representation Models
                            nr_topics=model_nr_topics,                      # Festlegung auf Zielwert der Topic-Anzahl
                            verbose=True,                                   # Fortschrittsanzeige
                            calculate_probabilities=True,                   # Topic-Wahrscheinlichkeiten
                            )
    except Exception as e:
        print(f"Fehler im Aufruf des BERTopic-Moduls für das Modell \"{model_name}\": {e}.")
        pass

    # Beginn des Clusterings der Themen
    topics = None
    probs = None
    try:
        print(f"\nDas Topic Modeling wird jetzt für das Modell \"{model_name}\" initiiert.\n")
        topics, probs = topic_model.fit_transform(docs, embeddings=embeddings)

    except Exception as e:
        if "max_df corresponds to < documents than min_df" in str(e):
            print(80*"=", f"\nBeim Model \"{model_name}\" ist in diesem Durchlauf der bekannte Fehler aufgetreten"
                  f": {e}.\nDie Funktion wird weiter ausgeführt.\n", 80*"=")
        else:
            print(f"Fehler im Topic Modeling für das Modell \"{model_name}\": {e}.")
        return None, None, None, None

    # Outlier autom. reduzieren lassen im Sinne einer nachträglichen Clusterzuordnung
    if topics is not None:
        try:
            new_topics = topic_model.reduce_outliers(docs, topics, strategy="distributions")
            topic_model.update_topics(docs, topics=new_topics, representation_model=representation_model)
            topics = new_topics

            # Speichern
            df_doc_topics = pd.DataFrame({"topics": topics})
            df_doc_topics.to_csv(topic_results_dir / f"doc_topics_assignment_{realm}_{model_name}_{datum}_{hdbscan_min_cluster_size}_{umap_n_neighbors}_{vec_max_df}_{vec_min_df}_{model_nr_topics}_{ai_model}.csv", encoding="utf-8")
            print("\nTopic-Dokumenten-Zuordnung wurde erfolgreich gespeichert.")

        except ValueError as e:
            print(f"Fehler in der Reduzierung der Outlier-Themen beim Modell \"{model_name}\": {e}.")
            new_topics = topics if topics is not None else None
            # Speichern
            df_doc_topics = pd.DataFrame({"topics": new_topics})
            df_doc_topics.to_csv(topic_results_dir / f"doc_topics_assignment_{realm}_{model_name}_{datum}_{hdbscan_min_cluster_size}_{umap_n_neighbors}_{vec_max_df}_{vec_min_df}_{model_nr_topics}_{ai_model}.csv", encoding="utf-8")
            print("Fehler! Die Topic-Dokumenten-Zuordnung wurde dennoch erfolgreich gespeichert.")
    elif topics is None:
            new_topics = None
            print(f"Das Topic Modeling für das Modell \"{model_name}\" konnte nicht durchgeführt werden. Die Topics sind None.")

    # Ergebnisse der Topics als Df einer Variable zuweisen
    topics_summary = topic_model.get_topic_info()

    # Zeit nehmen
    end_time = time.time()
    duration = round(int(((end_time - start_time)/60)), 0)

    # Kurze Auswertungen pro Durchgang ausgeben
    print(80*"=")
    print(f"\nAnalyseergebnisse für das Modell \"{str(model_name)}\" bei einer Laufzeit von {duration if duration > 0 else 1} Minuten:\n")
    print(f"1. Parameter:\n1.1 HDBSCAN min_cluster_size = {hdbscan_min_cluster_size},\n1.1 HDBSCAN min_sample_size ={hdbscan_min_samples},"
        f"\n1.2 UMAP n_neighbors = {umap_n_neighbors},"
        f"\n1.3 Vectorizer max_df = {vec_max_df},"
        f"\n1.4 Vectorizer min_df = {vec_min_df},\n1.5 BERTopic Model min_nr_topics = {model_nr_topics},\n1.6 AI Model = {ai_model}")
    print(f"2. Anzahl der gefundenen Topics = {len(topics_summary)-1}.") # Outlier werden nicht mit angegeben!
    print(f"3. Mittelwert der Publikationen pro Topic = {topics_summary["Count"].mean():.0f}")
    print(f"4. Relation der Outliers am Gesamtkorpus = {((topics_summary["Count"].iloc[0] / topics_summary["Count"].sum())*100):.2f} %.")
    print(f"5. Die Variable \"topics\" enthält die Topic-Zuordnungen pro Dokument und war nicht leer = {topics is not None}.")
    print(80*"=")

    # Ergebnisse pro Durchgang in xlsx und csv lokal speichern
    if save_xlsx is True:
        df = pd.DataFrame(topics_summary)
        df.to_excel(rf"{topic_results_dir}/topic_results_{realm}_{model_name}_{datum}_{hdbscan_min_cluster_size}_{umap_n_neighbors}_{vec_max_df}_{vec_min_df}_{model_nr_topics}_{ai_model}.xlsx", sheet_name="daten", engine="openpyxl")
        df.to_csv(rf"{topic_results_dir}/topic_results_{realm}_{model_name}_{datum}_{hdbscan_min_cluster_size}_{umap_n_neighbors}_{vec_max_df}_{vec_min_df}_{model_nr_topics}_{ai_model}.csv", encoding="utf-8")
        #os.startfile(rf"C:\Users\felix\OneDrive\Desktop\masterarbeit\01_data\03_topic_modeling\topic_results_{model_name}_{datum}_{hdbscan_min_cluster_size}_{umap_n_neighbors}_{vec_max_df}_{vec_min_df}_{model_nr_topics}_{ai_model}.xlsx")

    # Das trainierte Modell ggf. speichern, damit es immer wieder geladen werden kann
    full_path = models_dir / f"{realm}_{model_name}_{datum}_{hdbscan_min_cluster_size}_{umap_n_neighbors}_{vec_max_df}_{vec_min_df}_{model_nr_topics}_{ai_model}"

    # Modell speichern
    if save_model is True:
        try:
            topic_model.save(full_path,
                            serialization="safetensors",
                            save_ctfidf=True,
                            save_embedding_model=sentence_transformer)
        except Exception as e:
            print(f"Fehler beim Speichern des trainierten Modells {model_name}: {e}.")
            pass

    return probs, topics, topics_summary, topic_model


In [ ]:
###########################################################################
# Definition zweier Gensim-Coherence-Score-Funktionen als eines zentralen Gütekriterium für die Topic-Modelle
# (vgl. Röder Röder, Michael, Andreas Both, und Alexander Hinneburg. „Exploring the space of topic coherence measures“.
# Proceedings of the Eighth ACM International Conference on Web Search and Data Mining, 2015)
###########################################################################
# Dieser Code wurde mit Hilfe von Copilot vervollständigt [KI]

from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

def gensim_coherence(trained_topic_model: SentenceTransformer, documents: list[str]) -> float:
    """
    Berechnet den C-V-Score für ein trainiertes BERTopic-Modell.
    """

    topics = trained_topic_model.get_topics()

    topic_words = []
    for topic_id in topics:
        if topic_id != -1:
            words = [word for word, _ in trained_topic_model.get_topic(topic_id)[:10]]
            topic_words.append(words)

    splitted_docs = [doc.lower().split() for doc in documents]

    dict = Dictionary(splitted_docs)

    coherence_model = CoherenceModel(
        topics=topic_words,
        texts=splitted_docs,
        dictionary=dict,
        coherence="c_v"
    )

    score = coherence_model.get_coherence()

    return score

def gensim_coherence_npmi(trained_topic_model: SentenceTransformer, documents: list[str]) -> float:
    """
    Berechnet den C-NPMI-Score für ein trainiertes BERTopic-Modell.
    """

    topics = trained_topic_model.get_topics()

    topic_words = []
    for topic_id in topics:
        if topic_id != -1:
            words = [word for word, _ in trained_topic_model.get_topic(topic_id)[:10]]
            topic_words.append(words)

    splitted_docs = [doc.lower().split() for doc in documents]

    dict = Dictionary(splitted_docs)

    coherence_model = CoherenceModel(
        topics=topic_words,
        texts=splitted_docs,
        dictionary=dict,
        coherence="c_npmi"
    )

    score = coherence_model.get_coherence()

    return score


## Hyperparameter-Tuning und Grid Search

Ab der folgenden Zelle beginnt der _grid search_ im Sinne des _hyperparameter tunings_ im Rahmen einer eigens definierten Funktion.

Die Parameter, die in einem bestimmten Wertebereich durchlaufen werden, sind gelistet.

Die Ergebnisse aller Modelle und ihrer Parameter werden in einer Tabelle gelistet und gespeichert.

In [ ]:
###########################################################################
# Hyperparameter-Tuning
###########################################################################

# Erstellung der leeren Ergebnisliste
ergebnisse = []

#########################################################
# Hyperparameter-Tuning-Funktion
#########################################################

def hyperp_tuning(tuple_list: Annotated[tuple, "Hier werden fünf Variablen übergeben: embedding_model, docs, embeddings, model_name, datum"],
                  hdbscan_cluster_range: list[int] = [15],
                  hdbscan_sample_range: list[int] = [5],
                  umap_neighbor_range: list[int] = [10],
                  cv_mindf_range: list[int] = [4],
                  cv_maxdf_range: list[float] = [0.9]) -> pd.DataFrame:
    """
    Diese Funktion ist ein systematisches Hyperparamter-Tuning für die drei ausgewählten Sentence-Transformer und verschiedene
    Hyperparameter in den folgenden Algorithmen: HDBSCAN, UMAP.

    Args:
    -

    Returns:
    - Dataframe mit den Ergebnissen für alle Modelle und Hyperparameter-Ranges
    """

    for a,b,c,d,e in tuple_list:

        for umap in umap_neighbor_range:                    # UMAP n-neighbors alternieren lassen

            for hdbscan_cluster in hdbscan_cluster_range:   # HDBSCAN min_cluster_size alternieren lassen

                for hdbscan_sample in hdbscan_sample_range: # HDBSCAN sample_size alternieren lassen

                    for cv_mindf in cv_mindf_range:         # CountVectorizer alternieren lassen

                        for cv_maxdf in cv_maxdf_range:     # CountVectorizer alternieren lassen

                            probs, topics_all, topics, topic_model = topic_clustering(
                                                                                    realm="publications",
                                                                                    sentence_transformer=a,
                                                                                    docs=b,
                                                                                    embeddings=c,
                                                                                    stop_words=comprehensive_stopwords,
                                                                                    umap_n_neighbors=umap,
                                                                                    hdbscan_min_cluster_size=hdbscan_cluster,
                                                                                    hdbscan_min_samples=hdbscan_sample,
                                                                                    vec_min_df=cv_mindf,
                                                                                    vec_max_df=cv_maxdf,
                                                                                    save_xlsx = True,
                                                                                    save_model=False,
                                                                                    model_name=d,
                                                                                    datum=e)
                            if topics is not None:
                                try:
                                    ergebnisse.append({
                                        "model": str(d),
                                        "umap_n_neighbors": umap,
                                        "hdbscan_min_cluster_size": hdbscan_cluster,
                                        "hdbscan_min_samples_size": hdbscan_sample,
                                        "vectorizer_mind_df": cv_mindf,
                                        "vectorizer_max_df":cv_maxdf,
                                        "count_topics": (len(topics)-1),
                                        "relation_outliers": topics["Count"].iloc[0] / topics["Count"].sum(),
                                        "median_topic_cluster": topics["Count"][1:].median(),
                                        "average_topic_cluster": topics["Count"][1:].mean(),
                                        "topic_cluster_sizes": topics["Count"][1:].tolist(),
                                        "keywords_list": topics["Representation"][1:].tolist(),
                                        "topic_names": topics["Name"][1:].tolist(),
                                        "c_v_score": gensim_coherence(topic_model, b),
                                        "c_npmi_score": gensim_coherence_npmi(topic_model, b)
                                    })
                                except Exception as e:
                                    print(f"Fehler bei Modell {str(d)} während des Speicherns der Ergebnisse: {e}.")
                                    pass
                            else:
                                print("Die Topics sind leer oder fehlerhaft. Die Funktion geht zum nächsten Durchlauf.")

                                ergebnisse.append({
                                        "model": str(d),
                                        "umap_n_neighbors": umap,
                                        "hdbscan_min_cluster_size": hdbscan_cluster,
                                        "hdbscan_min_samples_size": hdbscan_sample,
                                        "vectorizer_mind_df": cv_mindf,
                                        "vectorizer_max_df":cv_maxdf,
                                        "count_topics": 0,
                                        "relation_outliers": 0,
                                        "median_topic_cluster": 0,
                                        "average_topic_cluster": 0,
                                        "topic_cluster_sizes": 0,
                                        "keywords_list": 0,
                                        "topic_names": 0,
                                        "c_v_score": 0,
                                        "c_npmi_score": 0
                                    })

In [ ]:
#############################################################################################
# Funktion aufrufen für Grid-Search resp. Hyperparameter-Tuning (wird nur durchgeführt, wenn die Pipeline ausgewählt ist)
#############################################################################################

if grid_search_pipeline is True:
    print("Es wird das Hyperparameter-Tuning (Grid-Search) für die Publikationsdaten gestartet.\n")

    #############################################################################################
    # Vorbereitung der Tuple-Liste für die Grid-Search
    ##############################################################################################

    # Tuple mit den wichtigsten Parametern der geladenen Basismodelle erstellen lassen
    tuples_four_models = []
    for x,y in model_dict.items():
        tuples_four_models.append((y.raw_instance, cleaned_docs, y.embeddings, y.name, datum))

    #############################################################################################
    # Grid-Search durchführen: Für aktuell 4 Modelle mit 5 Parametern und je 2-4 Werten ergeben sich 4*4*3*3*3*2 = 864 Loops!
    #############################################################################################

    # Zeit nehmen
    start_point = time.time()

    # Funktionsaufruf mit den festgelegten Hyperparameter-Wertebereichen
    try:
        hyperp_tuning(tuple_list=tuples_four_models,
                    hdbscan_cluster_range=[20, 40, 60, 80],
                    hdbscan_sample_range=[5, 10, 15],
                    umap_neighbor_range= [15, 30, 50],
                    cv_mindf_range=[2, 3, 5],
                    cv_maxdf_range=[0.90, 0.95])

    except Exception as e:
        print(f"\nFehler beim Hyper-Tuning: {e}.")
        traceback.print_exc()

    finally:
        # Ergebnisliste als Dataframe speichern und öffnen
        df_ergebnisse = pd.DataFrame(ergebnisse)

        df_ergebnisse.to_excel(rf"{grid_search_dir}/grid_search_results_pubs_{datum}.xlsx",
                            engine="openpyxl", sheet_name="tuning_results")
        df_ergebnisse.to_csv(rf"{grid_search_dir}/grid_search_results_pubs_{datum}.csv", encoding="utf-8")
        try:
            os.startfile(rf"{grid_search_dir}/grid_search_results_pubs_{datum}.xlsx")
        except Exception as e:
            print(f"Fehler beim Öffnen der Grid-Search-Ergebnisse: {e}.")
            pass

        # Laufzeit final ausgeben
        end_point = time.time()
        dur = round(int((end_point-start_point)/60), 0)
        print(f"Laufzeit des Grid-Search insgesamt etwa {dur} Minuten.")

else:
    print("Das Hyperparameter-Tuning (Grid-Search) für die Publikationsdaten wird hier nicht durchgeführt.\n")

### Auswertung und Evaluation des Grid-Search der Publikationen

Die Daten der verschiedenen Modellierungsdurchläufe werden anhand einer Überprüfung der verschiedenen Features ausgewertet, um eine Auswahl der besten Modelle / des besten Modells zu ermöglichen.

Zentral dafür sind die Kennzahlen zu den folgenden Bereichen:
- Anzahl der gefundenen Themencluster (sollte weder zu klein noch zu groß sein!)
- Relative Anzahl an nicht-zuordbaren Dokumenten (sollte möglichst klein sein!)
- Relative Anzahl einzigartiger Wörter in den Keyword-Listen (sollte einen hohen Wert haben, aber nicht zu hoch, um einerseits Themenredundanz zu vermeiden und andererseits nicht zu viele distrinkte Einzelthemen zu umfassen!)
- Berechneter C-V-Score für die _topic word coherence_
- Berechneter U-NPMI-Score  für die  _topic word coherence_

In [ ]:
# Einlesen der Ausgabedatei mit den Ergebnissen des Grid-Search, erste Auswertungen sowie Erstellung
# von einem neuen Feature: Keyword Uniquness

if grid_search_pipeline is True:
    print("Es wird die Ausgabedatei der Grid-Search für die Publikationsdaten eingelesen und ausgewertet.\n")

    # Anzeigeoptionen für Pandas anpassen
    pd.set_option("display.max_colwidth", None)
    pd.set_option("display.max_columns", None)

    # Letzte Grid-Search-Datei finden
    try:
        grid_files = [x for x in grid_search_dir.glob("*") if x.name.startswith("grid_search_results_pubs")]
        latest_grid_file = max(grid_files, key=lambda x: x.stat().st_mtime, default=None)

        df_grid_search = pd.read_excel(latest_grid_file, engine="openpyxl")

        print(f"Spalten des DataFrames: {df_grid_search.columns.tolist()}")
        # Output:
        # ['Unnamed: 0', 'model', 'umap_n_neighbors', 'hdbscan_min_cluster_size', 'hdbscan_min_samples_size', 'vectorizer_mind_df',
        # 'vectorizer_max_df', 'count_topics', 'relation_outliers', 'median_topic_cluster', 'average_topic_cluster',
        # 'topic_cluster_sizes', 'keywords_list', 'topic_names', 'c_v_score']
        print("Länge des Dataframes: ", len(df_grid_search))

    except Exception as e:
        print(f"Fehler beim Einlesen der Grid-Search-Ergebnisse: {e}.")
        pass

    # Umwandlung der Keywords-Listen von Strings in Listen-Objekte
    def list_conversion(x):
        """Liest die Strings aus und wandelt sie in verschachtelte Listen um."""

        if pd.isna(x) or x is None:
            return []

        if isinstance(x, str):
            return ast.literal_eval(x)

        return x

    # Anwendung der Funktion auf die Keyword-Spalte
    try:
        df_grid_search["keywords_list"] = df_grid_search["keywords_list"].apply(list_conversion)
    except Exception as e:
        print(f"Fehler bei der Umwandlung der Keywords-Listen: {e}.")
        pass


    # Funktion zur Berechnung der Einzigartigkeit der Keywords
    def uniqueness_of_keywords(keywords_list: list[list[str]]) -> float:
        """
        Diese Funktion berechnet den Anteil einzigartiger Keywords in einer Liste von Keyword-Listen.

        Args:
        - keywords_list = Liste von Listen mit Keywords pro Topic

        Returns:
        - Anteil einzigartiger Keywords als Float
        """

        if not isinstance(keywords_list, list):     # Sollte durch Vorverarbeitung mit "list_conversion" ausgeschlossen sein!
            return 0.0

        all_keywords = [x for sublist in keywords_list for x in sublist]
        unique_keywords = set(all_keywords)

        uniqueness_ratio = len(unique_keywords) / (len(all_keywords) if all_keywords else 0)

        return uniqueness_ratio

    # Anwendung der Funktion auf die Keywords-Spalte
    try:
        df_grid_search["keyword_uniqueness"] = df_grid_search["keywords_list"].apply(uniqueness_of_keywords)
    except Exception as e:
        print(f"Fehler bei der Berechnung der Keyword-Einzigartigkeit: {e}.")
        pass

    # Finaler Check der Ergebnisse
    try:
        print("Kurzer Check des finalen Df mit den neu erstellten Features.\n")

        # Auswertung des Grid-Search in einigen Kennzahlen nach folgenden Überlegungen:
        # 1. Zentral sind die verschiedenen Modelle
        # 2. Die Hyperparameter spielen zunächst keine primäre Rolle, sondern nur die Themenergebnisse
        # 3. count_topics: Anzahl der Topics ist kritisch und ein Gütemaß für das Modell
        # 4. relation_outliers: Ebenfalls kritisch, da das Ausmaß von them. Außenseitern Auskunft über die Breite der Thmene gibt
        # 5. topic_cluster_sizes: Werden genutzt, um zu überprüfen, wie gleichverteilt die Themen sind, was die Dokumente angeht
        # 6. keywords_list und topic_names: Werden genutzt, um zu überprüfen, wie viele wörtliche Überschneidungen es zwischen den Themen gibt

        df_grouped = df_grid_search.groupby("model").agg({
            "count_topics":["mean", "min", "max"],
            "relation_outliers":["mean", "min", "max"],
            #"topic_cluster_sizes":"modus",
            "c_v_score": ["mean", "min", "max"],
            "c_npmi_score": ["mean", "min", "max"],
            "average_topic_cluster": ["mean", "min", "max"],
            "keyword_uniqueness": ["mean", "min", "max"]
        })

        print(f"Es folgt eine aggregierte Übersicht der Ergebnisse nach den benutzten Modellen:\n\n{df_grouped}")
        print(80*"=")

    except Exception as e:
        print(f"Fehler bei der Ausgabe der Ergebnisse: {e}.")
        pass

else:
    print("Die Ausgabedatei der Grid-Search für die Publikationsdaten wird hier nicht eingelesen und ausgewertet.\n")


In [ ]:
# Auswertung durchführen mit einer Filterung nach den oben benannten Kriterien in zwei Schritten

if grid_search_pipeline is True:
      print("\nEs wird die Auswertung der Grid-Search-Ergebnisse mit Filterkriterien durchgeführt.\n")

      # Strenge Filterkriterien anwenden, um die besten Modelle zu identifizieren
      try:
            df_grid_search_filtered_strong = df_grid_search[(df_grid_search["count_topics"] > 15) &    # Die Anzahl der Themencluster sollte über 13 liegen
                                                (df_grid_search["keyword_uniqueness"] > 0.80) &        # Die Keywords sollten einen hohen Grad an Einzigartigkeit aufweisen
                                                (df_grid_search["relation_outliers"] < 0.05) &         # Die Relation der Outlier sollte unter 10 % liegen
                                                (df_grid_search["c_v_score"] > 0.65) &                 # Der c_v-Score sollte im oberen Bereich liegen, hier über 0.5
                                                (df_grid_search["c_npmi_score"] > -0.1)               # Der c_npmi-Score sollte nur über 0.0 liegen
                                                ]

            a1 = len(df_grid_search_filtered_strong[["model", "count_topics", "relation_outliers", "c_v_score", "keyword_uniqueness", "average_topic_cluster"]])
            print(f"Anzahl der verbleibenden Modelle nach Anwendung der harten Filterkriterien: {a1}.\n")
            print(df_grid_search_filtered_strong["model"].value_counts())
            df_grid_search_filtered_strong_sorted = df_grid_search_filtered_strong.sort_values(by=["c_v_score"], ascending=False)
            print(f"Die besten Modelle sind:\n\n{df_grid_search_filtered_strong_sorted}")
            print(80*"=")

            # Zum Gegentesten: Leichte Lockerung der Filterkriterien, um zu sehen, wie sich das auf die Modelle auswirkt
            df_grid_search_filtered_weak = df_grid_search[
                                                (df_grid_search["count_topics"] > 10) &             # Reduziert auf 10 Themen
                                                (df_grid_search["keyword_uniqueness"] > 0.75) &     # Reduziert auf 0.75
                                                (df_grid_search["relation_outliers"] < 0.1) &       # Reduziert auf 10 % Outlier
                                                (df_grid_search["c_v_score"] > 0.60) &              # Ein hoher c_v-Score ist relevant!
                                                (df_grid_search["c_npmi_score"] > -0.2)             # Reduziert auf 0.05
                                                ]

            print(f"\nModelle mit gelockerten Kriterien: insgesamt {len(df_grid_search_filtered_weak)} Modelle übrig.\n")
            print(df_grid_search_filtered_weak["model"].value_counts())

            # # Ranking der verbleibenden Modelle basierend auf dem c_v_score
            print("\nDa bei einer Lockerung der Filterkriterien viele Modelle übrig bleiben, wird nach dem c_v_score absteigend sortiert,"
                  " um zu erkennen, welche Modelle die Top 5 sind:\n")
            df_grid_search_filtered_weak_sorted = df_grid_search_filtered_weak.sort_values(by=["c_v_score"], ascending=False)
            print(df_grid_search_filtered_weak_sorted.head(10))

      except Exception as e:
            print(f"Fehler bei der Filterung der Grid-Search-Ergebnisse: {e}.")
            pass

else:
      print("Die Auswertung der Grid-Search-Ergebnisse mit Filterkriterien wird hier nicht durchgeführt.\n")

In [ ]:
# Das beste Modell wird ausgewählt für die weiteren Analysen

if grid_search_pipeline is True:
    print("\nDas beste Modell wird jetzt für die finale Analyse gespeichert.\n")

    # Das beste Modell extrahieren
    try:
        top_models = df_grid_search_filtered_strong.iloc[[0]]
        top_models.to_excel(rf"{grid_search_dir}/top_models_overview_pubs_{datum}.xlsx",
                            engine="openpyxl", sheet_name="top_models_pubs")

        print("\nTop-Modell bei den Publikationen:")
        print(top_models)
    except Exception as e:
        print(f"Fehler beim Speichern der Top-Modelle nach Excel: {e}.")
        pass

else:
    print("Das beste Modell für die finale Analyse wird hier nicht gespeichert.\n")

### Definition der besten/des besten Modelle/s

An dieser Stelle muss entschieden werden, ob das/die beste/n Modell/e in die Modellklasse geladen werden sollen oder nicht (dieser Schritt ist deswegen nicht automatisiert!). Es kann durchaus Gründe geben, mehrere Modelle in die Top-Model-Class zu laden. Üblicherweise wird aber nur das Modell auf Platz 1 genommen.

> _Grid Search_ und _Hyperparameter Tuning_ sind damit beendet!

Was folgt, ist der Durchlauf für die Evaluation und die Ausgabe der Ergebnisse.

---

### Zwischenfazit zu den Topics der vier Modelle

Der _grid search_ hat Folgendes ergeben:
-- TODO Tabelle einfügen --

Aufgrund der Metriken, die vor dem Hintergrund der domänenspezifischen Situation interpretiert wurden, ist deutlich geworden, dass nur die Modelle XX mit den Parametern XX zu sinnvollen Ergebnissen führen -- zumindest den Zahlen nach.
Diese Modelle wurden lokal gespeichert und werden im Folgenden noch einmal durchlaufen, um mit Unterstützung eines LLMs (dem Modell "sonnetxx" von Anthropic) über die Einbindung via LiteLM die entstandenen _topic cluster_ zu interpretieren und ihre Themencluster zu visualisieren.

Letztlich soll so ihre Verständlichkeit und Lesbarkeit erhöht werden.

## Evaluation auf Basis des besten/der besten Modells/Modelle

> Dafür muss die Variable in der zweiten Zelle neu gesetzt werden!

Damit der folgende Abschnitt durchlaufen kann, muss
(i) in der zweiten Zelle die Variable grid_search auf False gesetzt werden,
(ii) die Data Class top_model_dict mit dem Sentence Transformer eingestellt werden, für den man sich entschieden hat (in diesem Fall "all-Mini"

Die Codezeilen geben dann zunächst die Modellparameter aus und kreieren mit der leeren Instanz Embeddings und schließlich eine trainierte Instanz, die lokal gespeichert wird.  

In [ ]:
# Check der Werteauslese der besten Modelle

if evaluation_pipeline is True:
    print("\nDie Hyperparameter der/des Top-Modelle/Top-Modells werden jetzt ausgegeben.\n")

    #Finden der zuletzt gespeicherten Datei mit den Top-Modellen
    try:
        top_model_files = [x for x in grid_search_dir.glob("*") if x.name.startswith("top_models_overview_pubs")]
        latest_top_model_file = max(top_model_files, key=lambda x: x.stat().st_mtime, default=None)
        top_models = pd.read_excel(latest_top_model_file, engine="openpyxl")

        print("\nTop-Modell bei den Publikationen:")
        print(top_models)

    except Exception as e:
        print(f"Fehler beim finden der der letzten Top-Modelle in Excel: {e}.")
        pass

    try:
        for x in top_models["model"].values:
            aa=int(top_models[top_models["model"] == x]["hdbscan_min_cluster_size"].values[0]),
            bb=int(top_models[top_models["model"] == x]["hdbscan_min_samples_size"].values[0]),
            cc=int(top_models[top_models["model"] == x]["umap_n_neighbors"].values[0]),
            dd=int(top_models[top_models["model"] == x]["vectorizer_mind_df"].values[0]),
            ee=float(top_models[top_models["model"] == x]["vectorizer_max_df"].values[0])
            print(f"\n{x}: HDBSCAN min_cluster_size = {aa},"
            f" HDBSCAN min_sample_size = {bb}, UMAP n_neighbors = {cc},"
            f" Vectorizer min_df = {dd}, Vectorizer max_df = {ee}")
    except Exception as e:
        print(f"Fehler bei der Ausgabe der Hyperparameter der Top-Modelle: {e}.")
        pass

In [ ]:
#######################################################################
# Prüfung der Top-Modell-Dataclass mit anschließendem Aufruf der Embeddings
#######################################################################

if using_top_models is True:
    for x,y in top_model_dict.items():
        print(f"Die Top-Modell-Dataclass hält aktuell folgende Modelle:\n")
        print(y.name)

# Suchen und Laden der Embeddings -- falls vorhanden, ansonsten werden diese noch einmal
# erstellt

##########################################
# Funktion zum Check auf lokale Embeddings
##########################################

def check_local_embeddings_list(model_dict: dict) -> list:
    """Diese Funktion checkt, ob für alle Modelle im übergebenen Dict Embeddings
    lokal vorhanden sind.

    Args:
    - model_dict = Dict mit den Modellen

    Returns:
    - list: Gibt eine Liste der Modelle zurück, für die keine Embeddings gefunden werden
    konnten.
    """

    not_exist = []

    for x,y in model_dict.items():
        if y.embeddings is None:
            print(f"Für das Modell {y.name} sind keine lokalen Embeddings vorhanden.")
            not_exist.append(y.name)
        else:
            print(f"Für das Modell {y.name} sind lokale Embeddings vorhanden.")

    return not_exist

# Funktionsaufruf
if using_top_models is True:
    list_no_embedds = check_local_embeddings_list(top_model_dict)

    if not list_no_embedds:
        print("Alle Top-Modelle aus der Data Class haben lokale Embeddings geladen.")
    else:
        print("\nFolgende Modelle haben keine geladenen Embeddings: {}.".format(str(list_no_embedds)))
        print("\nDiese Embeddings werden noch einmal erstellt!")

        # Funktinausfruf zum Erstellen und Speichern der Embeddings oder zum Laden, falls Datei vorhanden
        create_or_load_embeddings("publications", cleaned_docs, top_model_dict, False)

In [ ]:
#######################################################################
# Funktionsuafurf des Topic CLusterings mit Einbindung von ChatGPT im Representation-Model
# Quelle: https://maartengr.github.io/BERTopic/getting_started/representation/llm.html#litellm
# (Anm.: Der Api-Key wird oben schon definiert!)
#######################################################################
# Folgende Schritte werden durchlaufen:
# 1. Modelle werden trainiert und gespeichert -- mit den besten Parametern aus dem Grid-Search und Anthropic als LLM im representation model
# 2. Grafische Auswertung der Topic Clusterings mithilfe der in Bertopic integrierten Visualisierungsfunktionen
# 3. Detaillierte Analyse der Topic Clusterings und deren Keywords

# Funktionsaufruf mit den besten Parametern und den beiden besten Modellen

if evaluation_pipeline is True:

    # Liste zur Speicherung der Ergebnisse der besten Modelle und Entscheidung über die Nutzung von OpenAI-Einbindung
    ergebnisse_best_models = []
    ai_model = "open_ai"   # "open_ai" oder None

    if using_top_models is True:                # Nur, wenn die besten Modelle genutzt werden sollen

        for x, y in top_model_dict.items():

            if y.embeddings is not None:        # Check, ob die Embeddings geladen wurden

                print(f"\nDas Topic Modeling wird jetzt mit dem Modell \"{y.name}\" und ChatGPT als LLM im Representation-Model durchgeführt.")

                try:

                    _, y.doc_topics_assignment, y.final_topics_df, y.trained_instance = topic_clustering(
                                                                                                realm="publications",
                                                                                                sentence_transformer=y.raw_instance,
                                                                                                docs=cleaned_docs,
                                                                                                embeddings=y.embeddings,
                                                                                                stop_words=comprehensive_stopwords,
                                                                                                hdbscan_min_cluster_size=int(top_models[top_models["model"] == y.name]["hdbscan_min_cluster_size"].values[0]),
                                                                                                hdbscan_min_samples=int(top_models[top_models["model"] == y.name]["hdbscan_min_samples_size"].values[0]),
                                                                                                umap_n_neighbors=int(top_models[top_models["model"] == y.name]["umap_n_neighbors"].values[0]),
                                                                                                vec_min_df=int(top_models[top_models["model"] == y.name]["vectorizer_mind_df"].values[0]),
                                                                                                vec_max_df=float(top_models[top_models["model"] == y.name]["vectorizer_max_df"].values[0]),
                                                                                                save_xlsx=True,
                                                                                                model_name=y.name,
                                                                                                datum=datum,
                                                                                                ai_model=ai_model,
                                                                                                save_model=True)

                except Exception as e:
                    print(f"Fehler beim Topic Modeling mit dem Modell {y.name} und ChatGPT als LLM: {e}.")
                    pass

                if y.final_topics_df is not None:
                    try:
                        ergebnisse_best_models.append({
                            "model": str(y.name),
                            "umap_n_neighbors": int(top_models[top_models["model"] == y.name]["umap_n_neighbors"].values[0]),
                            "hdbscan_min_cluster_size": int(top_models[top_models["model"] == y.name]["hdbscan_min_cluster_size"].values[0]),
                            "hdbscan_min_samples_size": int(top_models[top_models["model"] == y.name]["hdbscan_min_samples_size"].values[0]),
                            "vectorizer_mind_df": int(top_models[top_models["model"] == y.name]["vectorizer_mind_df"].values[0]),
                            "vectorizer_max_df": float(top_models[top_models["model"] == y.name]["vectorizer_max_df"].values[0]),
                            "count_topics": (len(y.final_topics_df)-1),
                            "relation_outliers": y.final_topics_df["Count"].iloc[0] / y.final_topics_df["Count"].sum(),
                            "median_topic_cluster": y.final_topics_df["Count"][1:].median(),
                            "average_topic_cluster": y.final_topics_df["Count"][1:].mean(),
                            "topic_cluster_sizes": y.final_topics_df["Count"][1:].tolist(),
                            "keywords_list": y.final_topics_df["Representation"][1:].tolist(),
                            "topic_names": y.final_topics_df["Name"][1:].tolist(),
                            "topic_names_AI": y.final_topics_df["ChatGPT_titles"][1:].tolist() if ai_model == "open_ai" else None,
                            "topic_summaries_AI": y.final_topics_df["ChatGPT"][1:].tolist() if ai_model == "open_ai" else None,
                            # "topic_names_KeyBERT": y.final_topics_df["Main"][1:].tolist(),
                            "c_v_score": gensim_coherence(y.trained_instance, cleaned_docs),
                            "c_npmi_score": gensim_coherence_npmi(y.trained_instance, cleaned_docs)
                        })
                    except Exception as e:
                        print(f"Fehler bei Modell {str(y.name)} während des Speicherns der Ergebnisse: {e}.")
                        pass
                else:
                    print("Die Topics sind leer oder fehlerhaft. Die Funktion geht zum nächsten Durchlauf.")

                    ergebnisse_best_models.append({
                            "model": str(y.name),
                            "umap_n_neighbors": int(top_models[top_models["model"] == y.name]["umap_n_neighbors"].values[0]),
                            "hdbscan_min_cluster_size": int(top_models[top_models["model"] == y.name]["hdbscan_min_cluster_size"].values[0]),
                            "hdbscan_min_samples_size": int(top_models[top_models["model"] == y.name]["hdbscan_min_samples_size"].values[0]),
                            "vectorizer_mind_df": int(top_models[top_models["model"] == y.name]["vectorizer_mind_df"].values[0]),
                            "vectorizer_max_df": float(top_models[top_models["model"] == y.name]["vectorizer_max_df"].values[0]),
                            "count_topics": 0,
                            "relation_outliers": 0,
                            "median_topic_cluster": 0,
                            "average_topic_cluster": 0,
                            "topic_cluster_sizes": 0,
                            "keywords_list": 0,
                            "topic_names": 0,
                            "topic_names_AI": 0,
                            "topic_summaries_AI": 0,
                            # "topic_names_KeyBERT": 0,
                            "c_v_score": 0,
                            "c_npmi_score": 0
                        })

                print(80*"=")

            else:
                print(f"Die Embeddings für das Modell {y.name} wurden nicht geladen.")
                pass

        # Ergebnisse der besten Modelle speichern
        df_ergebnisse_best_models = pd.DataFrame(ergebnisse_best_models)
        df_ergebnisse_best_models.to_excel(rf"{grid_search_dir}/best_models_results_{datum}.xlsx",
                            engine="openpyxl", sheet_name="best_model_results")
        try:
            os.startfile(rf"{grid_search_dir}/best_models_results_{datum}.xlsx")
        except Exception as e:
            print(f"Fehler beim Öffnen der Exceldatei! Evtl. OS-Modul vorhanden.")
    else:
        print("Die besten Modelle wurden nicht geladen! BItte erste Zelle prüfen.")

    # Checks
    print(f"Länge der Docs: {len(cleaned_docs)}.")
    for x,y in top_model_dict.items():
        try:
            print(f"Länge der Topics: {len(y.doc_topics_assignment)}.")
        except Exception as e:
            print(f"Fehler in der Auswertung der Länge: {e}.")

### Graphische Auswertungen der Publikationen

Die ausgewählten Modelle sind erstellt, gespeichert und in der Modellklasse geladen.

In diesem Teil wird eine Funktion erstellt und dann aufgerufen, die automatisch verschiedenen Visualisierungen der Topic-Cluster erstellt, ausgibt und lokal speichert.

Eine Interpretation der Ergebnisse erfolgt zum Abschluss des Notebooks.

In [ ]:
# Check auf korrekte befüllte Modellklassen

if evaluation_pipeline is True:
    print("\nEs wird ein Check der befüllten Modellklassen der Top-Modelle durchgeführt.\n")

    try:
        for x, y in top_model_dict.items():
            print(f"\nModell: {y.name}")
            print(f"Document-Topic-Zuweisungen: {type(y.doc_topics_assignment)}")
            print(f"Länge der Document-Topic-Zuweisung: {len(y.doc_topics_assignment)}")
            print(f"Finales Topic-Df: {type(y.final_topics_df)}")
            print(f"Länge des finalen Topic-Df: {len(y.final_topics_df)-1}") # Outlier müssen herausgenommen werden!
            print(f"Trainiertes Modell: {type(y.trained_instance)}")
    except Exception as e:
        print(f"Fehler beim Check der Modellklassen: {e}.")
        pass

In [ ]:
###########################################################
# Funktion definieren für insg. sieben Visualisierungen
###########################################################

def visuals_per_topic_model(
    realm: str,
    topic_model: Annotated[object, "Hier muss das trainierte topic.model übergeben werden"],
    docs: list[str],
    top_n_topics: int = 10,
    n_words: int = 10,
    model_name: str | None = None,
    embeddings: np.ndarray | None = None,
    sample_documents: list | None = None,
    visualizations: Annotated[list[str], "Eine Auswahl aus heatmap, topics, barchart, term_rank, hierarchy, documents, hierarchical_documents]"] = None,
    show_visuals: bool = False
) -> dict:

    """
    Sammelfunktion, um die graphischen Auswertungen zu den Topic-Clustern zu erstellen und lokal zu speichern!

    Args:
    - ...

    Returns:
    - Dict der erstellen Plots
    """

    # Ordner Set Up mit Pathlib
    current_dir = Path.cwd()
    target_folder = current_dir.parent / "01_data" / "03_topic_modeling"
    topic_visuals_folder = target_folder / "03_topic_visuals"

    # Zielordner checken (relativ zum Notebook)
    try:
        if Path(topic_visuals_folder).exists():
            print(f"Ordner \"{str(topic_visuals_folder)}\" vorhanden.")
        else:
            print(f"Ordner \"{str(topic_visuals_folder)}\" nicht vorhanden. Der Ordner wird im folgenden Schritt erstellt werden.")
            topic_visuals_folder.mkdir(parents=True)
    except Exception as e:
        print(f"Aufgrund eines Fehlers konnte nicht weiter verfahren werden: {e}.")
    print(80*"=")

    # Dict initialisieren, um die Visualisierungen zu speichern!
    figures = {}

    if visualizations is None:
        visualizations = ["heatmap", "topics", "barchart", "term_rank",
                         "hierarchy", "documents", "hierarchical_documents"]

    ###########################################################################
    # 1. Topic Heatmap
    ###########################################################################
    if "heatmap" in visualizations:
        try:
            print("1. Topic Heatmap erstellen.")

            # Plot erstellen!
            fig = topic_model.visualize_heatmap()
            # Plot dem Dict zuweisen
            figures["heatmap"] = fig

            # Ergebnis speichern
            path = rf"{topic_visuals_folder}/heatmap_{realm}_{model_name}_{datum}"
            fig.write_html(rf"{path}.html")
            try:
                fig.write_image(rf"{path}.png")
            except Exception as e:
                print(f"PNG-Export fehlgeschlagen (kaleido Version ist vorhanden!): {e}")
            # Plot ausgeben lassen
            if show_visuals is True:
                fig.show()

            print(f"Plot gespeichert: \"{path}\"")
        except Exception as e:
            print(f"Fehler \"{e}\".")

    ###########################################################################
    # 2. 2D Topic Map
    ###########################################################################
    if "topics" in visualizations:
        try:
            print("2. Topic Map erstellen.")

            # Plot erstellen
            fig = topic_model.visualize_topics(width=1200, height=800)
            # Plot dem Dict zuweisen
            figures["topics"] = fig

            # Ergebnis speichern
            path = rf"{topic_visuals_folder}/topic-map_{realm}_{model_name}_{datum}.html"
            fig.write_html(path)
            try:
                fig.write_image(path.replace(".html", ".png"))
            except Exception as e:
                print(f"PNG-Export fehlgeschlagen (kaleido 0.2.1 ist aber vorhanden!): {e}")

            # Plot anzeigen lassen
            if show_visuals is True:
                fig.show()

            print(f"Plot gespeichert: \"{path}\"")
        except Exception as e:
            print(f"Fehler \"{e}\".")

    ###########################################################################
    # 3. Topic Words Bar Chart
    ###########################################################################
    if "barchart" in visualizations:
        try:
            print("3. Topic Barchart erstellen.")

            # Plot erstellen
            fig = topic_model.visualize_barchart(
                top_n_topics=top_n_topics,
                n_words=n_words,
                height=500
            )
            # Plot dem Dict zuweisen
            figures["barchart"] = fig

            # Ergebnis speichern
            path = rf"{topic_visuals_folder}/bar-chart-words_{realm}_{model_name}_{datum}.html"
            fig.write_html(path)
            try:
                fig.write_image(path.replace(".html", ".png"))
            except Exception as e:
                print(f"PNG-Export fehlgeschlagen (kaleido 0.2.1 ist aber vorhanden!): {e}")

            # Plot anzeigen lassen
            if show_visuals is True:
                fig.show()

            print(f"Plot gespeichert: \"{path}\"")
        except Exception as e:
            print(f"Fehler \"{e}\".")

    ###########################################################################
    # 4. Term Rank Plot
    ###########################################################################
    if "term_rank" in visualizations:
        try:
            print("4. Term Ranking erstellen")

            # Plot erstellen
            fig = topic_model.visualize_term_rank(log_scale=True)
            # Plot dem Dict zuweisen
            figures["term_rank"] = fig

            # Plot speichern und anzeigen lassen
            path = rf"{topic_visuals_folder}/term-rank_{realm}_{model_name}_{datum}.html"
            fig.write_html(path)
            try:
                fig.write_image(path.replace(".html", ".png"))
            except Exception as e:
                print(f"PNG-Export fehlgeschlagen (kaleido 0.2.1 ist aber vorhanden!): {e}")

            # Plot anzeigen lassen
            if show_visuals is True:
                fig.show()

            print(f"Plot gespeichert: \"{path}\"")
        except Exception as e:
            print(f"Fehler \"{e}\".")

    ###########################################################################
    # 5. Hierarchical Topic Structure
    ###########################################################################
    if "hierarchy" in visualizations:
        try:
            print("5. Topic Hierarchy erstellen.")

            # Hierarchie erzeugen
            hierarchical_topics = topic_model.hierarchical_topics(docs)

            # Plot erstellen lassen
            fig = topic_model.visualize_hierarchy(
                hierarchical_topics=hierarchical_topics,
                width=1200,
                height=800
            )

            # Plot dem Dict zuweisen
            figures["hierarchy"] = fig

            path = rf"{topic_visuals_folder}/hierarchy_{realm}_{model_name}_{datum}.html"
            fig.write_html(path)
            try:
                fig.write_image(path.replace(".html", ".png"))
            except Exception as e:
                print(f"PNG-Export fehlgeschlagen (kaleido 0.2.1 ist aber vorhanden!): {e}")

            # Plot anzeigen lassen
            if show_visuals is True:
                fig.show()

            print(f"Gespeichert: \"{path}\".")

            # Speichern:
            tree = topic_model.get_topic_tree(hierarchical_topics)
            tree_path = rf"{topic_visuals_folder}/topic-tree_{realm}_{model_name}_{datum}.html"
            with open(tree_path, "w", encoding="utf-8") as f:
                f.write(tree)
                try:
                    fig.write_image(path.replace(".html", ".png"))
                except Exception as e:
                    print(f"PNG-Export fehlgeschlagen (kaleido 0.2.1 ist aber vorhanden!): {e}")
            print(f"Gespeichert: \"{tree_path}\".")

        except Exception as e:
            print(f"Fehler: {e}")

    ###########################################################################
    # 6. Document Distribution
    ###########################################################################
    if "documents" in visualizations:
        if embeddings is not None:
            try:
                print("6. Dokumenten-Karte erstellen.")

                # Plot erstellen
                fig = topic_model.visualize_documents(
                    docs,
                    embeddings=embeddings,
                    sample=sample_documents,
                    width=1500,
                    height=1000
                )

                # Plot dem Dict zuweisen
                figures["documents"] = fig

                path = rf"{topic_visuals_folder}/doc-map-2D_{realm}_{model_name}_{datum}.html"
                fig.write_html(path)
                try:
                    fig.write_image(path.replace(".html", ".png"))
                except Exception as e:
                    print(f"PNG-Export fehlgeschlagen (kaleido 0.2.1 ist aber vorhanden!): {e}")

                # Plot anzeigen lassen
                if show_visuals is True:
                    fig.show()

                print(f"Gespeichert: \"{path}\".")

            except Exception as e:
                print(f"Fehler: {e}")
        else:
            print("6. Da keine Embeddings übergeben wurden, konnte nicht geplottet werden.")

    ###########################################################################
    # 7. Hierarchical Document Visualization
    ###########################################################################
    if "hierarchical_documents" in visualizations:
        if embeddings is not None:
            try:
                print("7. Dokumenten-Karte in 2D nach Hierarchie erstellen.")

                # Check, ob die Hierarchie schon erstellt wurde
                if "hierarchy" not in figures:
                    hierarchical_topics = topic_model.hierarchical_topics(docs)

                # Plot erstellen
                fig = topic_model.visualize_hierarchical_documents(
                    docs,
                    hierarchical_topics=hierarchical_topics,
                    embeddings=embeddings,
                    sample=sample_documents,
                    width=1500,
                    height=1000
                )

                # Plot dem Dict zuweisen
                figures["hierarchical_documents"] = fig

                # Plotten und anzeigen lassen
                path = rf"{topic_visuals_folder}/hier-doc-map-2D_{realm}_{model_name}_{datum}.html"
                fig.write_html(path)
                try:
                    fig.write_image(path.replace(".html", ".png"))
                except Exception as e:
                    print(f"PNG-Export fehlgeschlagen (kaleido >=1.0.0 ist aber vorhanden!): {e}")

                # Plot anzeigen lassen
                if show_visuals is True:
                    fig.show()

                # Speichern
                print(f"Gespeichert: \"{path}\".")

            except Exception as e:
                print(f"Fehler: \"{e}\".")

        else:
            print("7. Da keine Embeddings übergeben wurden, konnte nicht geplottet werden.")

    print(f"\nAuswertung: Es wurden insgesamt {len(figures)} Plots erstellen und in folgendem Ordner gespeichert: {topic_visuals_folder}.\n")

    return figures


In [ ]:
###########################################################
# Visualisierungsfunktion aufrufen für das beste Modell
# (Es kann sinnvoll sein, vorzuentscheiden, ob die Visualisierungen
#  erstellt werden sollen oder nicht, da dies Zeit in Anspruch nimmt [s. Parameter show_visuals]!)
###########################################################

if evaluation_pipeline is True:
    print("\nDie Visualisierungen für die Top-Modelle werden jetzt erstellt und gespeichert.\n")

    # Dict initialisieren, um die Visualisierungen zu speichern
    figures_dict = {}

    for x,y in top_model_dict.items():

        figures_pubs = visuals_per_topic_model(
                                realm="publications",
                                topic_model=y.trained_instance,
                                docs=cleaned_docs,
                                embeddings=y.embeddings,
                                model_name=y.name,
                                show_visuals=False
                            )

        figures_dict[y.name] = figures_pubs

## Dynamisches Topic Modeling

In [ ]:
###########################################################
# Dynamisches Topic Modeling nach Jahren
# Quelle: https://maartengr.github.io/BERTopic/getting_started/topicsovertime/topicsovertime.html#bins
###########################################################

# Vorverarbeitung der Jahresdaten, angepasst an die unterschiedlichen, in den Rohdaten vorhandenen Formate
# Diese Funktion wurde mit Hilfe von Copilot komplettiert und dann erfolgreich gestestet! [KI]
def extract_year(date_str):
    """Hier werden die unterschiedlichen Formate, wie sie in den Daten vorhanden waren, passend aufbereitet."""

    date_str = str(date_str).strip()

    if len(date_str) == 4 and date_str.isdigit():
        return int(date_str)

    if "-" in date_str:
        return int(date_str.split("-")[0])

    if "." in date_str:
        return int(date_str.split(".")[-1])

    return None


# Datenvorvereitung mit den Jahresdaten mit dem ursprünglichen Dataframe
if evaluation_pipeline is True:
    print("\nDie Jahresdaten für das dynamische Topic Modeling werden jetzt vorbereitet.\n")

    df_all_copy = df_all.copy()
    print(df_all_copy.columns)

    df_all_copy["publication_year"] = df_all_copy["publication_date"].apply(extract_year)
    df_all_copy["publication_year"] = df_all_copy["publication_year"].astype(int)
    df_all_copy.loc[df_all_copy["publication_year"] == 1987, "publication_year"] = 2016
    df_all_copy.loc[df_all_copy["publication_year"] == 2013, "publication_year"] = 2015
    df_all_copy.loc[df_all_copy["publication_year"] == 2014, "publication_year"] = 2015
    date_list = df_all_copy["publication_year"].tolist()

    df_aggre = df_all_copy.groupby("publication_year")["title"].count()
    print(df_aggre)

    # Check der Längen
    print(f"Dokumente zu Datumsangaben passend = {len(date_list) == len(cleaned_docs)}.")

    print(f"Länge der Jahresangaben = {len(date_list)}.")
    print(f"Jahreszeitraum: {min(date_list)} - {max(date_list)}")
    print(f"Verteilung der Jahresdaten:\n{sorted(set(date_list))}")

In [ ]:
# Aufruf der BERTopic-Funktion für dynamisches Topic Modeling mit den besten Modellen
# (Anm.: Die Berechnung dauert bis zu 25 Minuten für zwei Graphen!)

if evaluation_pipeline is True:
    print("\nDas dynamische Topic Modeling für die Top-Modelle wird jetzt durchgeführt und die Graphen werden ausgegeben.\n")

    for x, y in top_model_dict.items():
        topics_over_time = y.trained_instance.topics_over_time(
                                                        docs=cleaned_docs,
                                                        timestamps=date_list,
                                                        #datetime_format="%Y",
                                                        nr_bins=len(set(date_list))
                                                        )

        fig = y.trained_instance.visualize_topics_over_time(topics_over_time)
        fig.write_html(rf"{topic_visuals_dir}/dynamic_topic_modeling_{y.name}_{datum}.html")
        try:
            fig.write_image(rf"{topic_visuals_dir}/dynamic_topic_modeling_{y.name}_{datum}.png")
        except Exception as e:
            print(f"PNG-Export fehlgeschlagen: {e}")

        print(f"Graph für {y.name} wurde erfolgreich gespeichert.")

else:
    print("Das dynamische Topic Modeling wird hier nicht durchgeführt.\n")


## Rückschreibung der Ergebnisse an die PIs

In [ ]:
# Gesamt-Dataframe erweitern: Erstellte Topics werden Publikationen/Drittmitteln
# und PIs zugeordnet.

if evaluation_pipeline is True:
    print("\nDie erstellten Topics der Top-Modelle werden jetzt den Publikationen im Dataframe zugeordnet.\n")

    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.expand_frame_repr', False)

    ##########################################################################################
    # Publikationen
    ##########################################################################################

    # 1.1 Themencluster zuordnen in den ursprünglichen Dataframe. D.h., dass die im Df vorhanden Publikationen
    # die zugehörigen Topic-Nummern erhalten
    print(df_all.columns)
    print(f"Länge des Dataframes: {len(df_all)}.")
    print(f"Länge der Dokumente: {len(cleaned_docs)}.")

    # Über Modelclass iterieren (nur die besten Modelle)
    for x,y in top_model_dict.items():
        print(f"\nDokument-Topic-Zuweisungen für Modell \"{y.name}\" werden dem Dataframe hinzugefügt.")
        df_all[f"topic_assignment_{y.name}"] = y.doc_topics_assignment

    # 1.2 Themen und Keywords zuordnen in den ursprünglichen Dataframe
    for x,y in top_model_dict.items():

        # Topic-Dataframe holen
        y.final_topics_df = y.trained_instance.get_topic_info()

        # Dicts erstellen, die Topic-Namen und Keywords zu den Topic-Nummern zuordnen
        print(f"\nThemen und Keywords für Modell \"{y.name}\" werden dem Dataframe hinzugefügt.")
        topic_names = y.final_topics_df.set_index("Topic")["Name"].to_dict()
        topic_keywords = y.final_topics_df.set_index("Topic")["Representation"].to_dict()

        df_all[f"topic_name_{y.name}"] = df_all[f"topic_assignment_{y.name}"].map(topic_names)
        df_all[f"topic_keywords_{y.name}"] = df_all[f"topic_assignment_{y.name}"].map(topic_keywords)

    print(f"\nNeue Spalten Publikationen: {df_all.columns}.")
    df_all.head()

else:
    print("Die erstellten Topics der Top-Modelle werden hier nicht den Publikationen zugeordnet.\n")

In [ ]:
# Nachdem nun die Ergebnisse des Topic Modelings der Publikationen vorliegen und damit auch den
# PIs zugeordnet wurden, erfolgt die aggregierte Informationsauswertung. Diese Auswertung wiederum
# wird dem PI-Dataframe hinzugefügt und gespeichert.

# Der Auswertungsfokus liegt auf den Fragen:
# 1. Wie viele Topics sind den jeweiligen PIs zugeordnet (Themendiversität)?
# 2. Wie ist die Verteilung der Topics bei den jeweiligen PIs? Und was sind die drei Hauptthemen?
##########################################################################################
# Publikationen (Diese Auswertung wurde durch die Hilfe von Copilot reduziert und optimiert! [KI])
##########################################################################################

if evaluation_pipeline is True:
    print("\nDie aggregierten Metriken für PIs aus den Publikationen werden jetzt erstellt und gespeichert.\n")

    for x,y in top_model_dict.items():
        print(f"\nAggregierte Metriken für PIs aus Publikationen werden für Modell \"{y.name}\" erstellt.\n")

        # Anonymisierung der PI-Namen im Dataframe
        pi_df = pd.read_csv(rf"{base_dir}/01_data/01_csv_data/00_pi_basics/FINALLY_ALL_pi_data.csv", encoding="utf-8")
        pi_hash_dict = pi_df.set_index("nachname")["pi_name_hashed"].to_dict()
        
        df_all["source"] = df_all["source"].map(pi_hash_dict)

        # Aggregation der Metriken pro PI
        pi_metrics = df_all.groupby("source").agg(
            topic_count=(f"topic_assignment_{y.name}", "nunique"),
            total_pubs=(f"topic_assignment_{y.name}", "size")
        ).reset_index()

        # Top 3 Themen
        pi_metrics["top3_topics_nr"] = df_all.groupby("source")[f"topic_assignment_{y.name}"].apply(
            lambda x: x.value_counts().head(3).index.tolist()).values
        pi_metrics["top3_topics_title"] = df_all.groupby("source")[f"topic_name_{y.name}"].apply(
            lambda x: [x for x in x.value_counts().head(3).index.tolist()]).values

        # Speichern als csv
        pi_metrics.to_csv(rf"{topic_results_dir}/pi_publication_metrics_{y.name}_{datum}.csv", index=False, encoding="utf-8")
        pi_metrics.to_excel(rf"{topic_results_dir}/pi_publication_metrics_{y.name}_{datum}.xlsx",
                            engine="openpyxl", sheet_name="pi_metrics")

    print(pi_metrics)

else:
    print("Die aggregierten Metriken für PIs aus den Publikationen werden hier nicht erstellt und gespeichert.\n")

## Abschluss

Die Ergebnisse dieser Analyse werden in der Arbeit dargestellt und im Kontext der Forschungsfragen bewertet.